# Scratchwork #1 - Quaternion Integers

In [1]:
import numpy as np
import random as rnd
import math
from gaussians import Zi

def split_array(arr):
    n = len(arr) // 2
    return arr[:n], arr[n:]

class Hi:
    """Integer-valued quaternions ('Gaussian quaternions')"""

    def __init__(self, z1: (Zi, np.ndarray, int) = Zi(), z2: (Zi, int) = Zi(), z3: int = 0, z4: int = 0):
        if isinstance(z1, Zi):
            self.__real = z1
            self.__imag = z2
            self.__arr = np.array([z1.real, z1.imag, z2.real, z2.imag], dtype=np.int64)
        elif isinstance(z1, np.ndarray):
            self.__arr = z1
            arr1, arr2 = split_array(z1)
            self.__real = Zi.from_array(arr1)
            self.__imag = Zi.from_array(arr2)
        elif isinstance(z1, (int, np.int64)):
            self.__real = Zi(z1, z2)
            self.__imag = Zi(z3, z4)
            self.__arr = np.array([z1, z2, z3, z4], dtype=np.int64)
        else:
            raise ValueError(f"{type(z1)} not supported")
            
    @property
    def real(self):
        return self.__real

    @property
    def imag(self):
        return self.__imag

    @property
    def array(self):
        return self.__arr

    @property
    def conjugate(self):
        # a, b, c, d = self.array
        # return Hi(a, -b, -c, -d)
        return Hi(self.real.conjugate, -self.imag)

    @property
    def norm(self) -> int:
        tmp = self * self.conjugate
        return int(tmp.array[0])
    
    def show(self):
        return f"Hi({repr(self.__real)}, {repr(self.__imag)})"

    def __abs__(self) -> float:
        return math.sqrt(self.norm)

    def __repr__(self) -> str:
        a, b, c, d = self.array
        return f"Hi({a}, {b}, {c}, {d})"

    def __str__(self) -> str:
        a, b, c, d = self.array
        return f"({a} + {b}i + {c}j + {d}k)"

    def __add__(self, other):
        return Hi(self.array + other.array)

    def __sub__(self, other):
        return Hi(self.array - other.array)

    def __neg__(self):
        return Hi(-self.array)

    def __eq__(self, other):
        return self.array == other.array

    def __ne__(self, other):
        return self.array != other.array

    def __mul__(self, other):
        """Multiplication according to the Cayley-Dickson construction"""
        a = self.real
        b = self.imag
        c = other.real
        d = other.imag
        # (a, b) * (c, d) = (a * c - d.conj * b, d * a + b * c.conj)
        z1 = a * c - d.conjugate * b
        z2 = d * a + b * c.conjugate
        return Hi(z1, z2)
    
    @staticmethod
    def hamilton_product(q1, q2):
        """Multiplication according to the classic Hamilton product"""
        a1, b1, c1, d1 = q1.array
        a2, b2, c2, d2 = q2.array
        # See https://en.wikipedia.org/wiki/Quaternion#Hamilton_product
        a = a1 * a2 - b1 * b2 - c1 * c2 - d1 * d2
        b = a1 * b2 + b1 * a2 + c1 * d2 - d1 * c2
        c = a1 * c2 - b1 * d2 + c1 * a2 + d1 * b2
        d = a1 * d2 + b1 * c2 - c1 * b2 + d1 * a2
        return Hi(a, b, c, d)

    def scalar_mul(self, scalar):
        """Multiply this quaternion by a scalar integer.

        Round the scalar to the nearest integer if necessary.
        """
        return Hi(round(scalar) * self.array)

    def __floordiv__(self, other):
        """Implements the // operator using 'round', instead of 'floor'."""
        numer = self * other.conjugate
        denom = other.norm
        quotient = np.round(numer.array / denom)
        return Hi(quotient.astype(np.int64))

    def to_gaussian_ints(self):
        """Convert this quaternion into two Gaussian integers"""
        a, b, c, d = self.array
        return Zi(int(a), int(b)), Zi(int(c), int(d))

    @staticmethod
    def random(low=-100, high=100):
        return Hi(np.array([rnd.randint(low, high) for _ in range(4)]))

    @staticmethod
    def modified_divmod(a, b):
        """Returns q & r, such that a = b * q + r, where
        r.norm < b.norm / 2
        """
        q = a // b
        r = a - b * q
        return q, r

## Random Quaternion Integers:

In [2]:
import random as rnd

rnd.seed(10)
quads = [Hi.random() for _ in range(5)]
h1, h2, h3, h4, h5 = quads

In [3]:
h1, h2, h3, h4, h5

(Hi(46, -92, 9, 23),
 Hi(47, -97, -48, 18),
 Hi(25, -29, 67, -59),
 Hi(-92, 33, 25, -17),
 Hi(-81, -37, 90, -8))

In [4]:
h1.show()

'Hi(Zi(46, -92), Zi(9, 23))'

In [5]:
Hi(Zi(46, -92), Zi(9, 23))

Hi(46, -92, 9, 23)

In [6]:
Hi(46, -92, 9, 23)

Hi(46, -92, 9, 23)

In [7]:
print(Hi(46, -92, 9, 23))

(46 + -92i + 9j + 23k)


## Print Quaternion Integers

In [8]:
for quad in quads:
    print(quad)

(46 + -92i + 9j + 23k)
(47 + -97i + -48j + 18k)
(25 + -29i + 67j + -59k)
(-92 + 33i + 25j + -17k)
(-81 + -37i + 90j + -8k)


## Conjugation, Norm, & Abs

In [9]:
h1.conjugate

Hi(46, 92, -9, -23)

In [10]:
h1.norm

11190

In [11]:
abs(h1)

105.78279633286313

## Quaternion Integer Arithmetic

In [12]:
h1 + h2

Hi(93, -189, -39, 41)

In [13]:
h1 - h2

Hi(-1, 5, 57, 5)

In [14]:
h1 * h2

Hi(-6744, -7520, -2360, 7198)

In [15]:
Hi.hamilton_product(h1, h2)

Hi(-6744, -7520, -2360, 7198)

Multiply a quaternion by a scalar (integer)

In [16]:
h1.scalar_mul(2)

Hi(92, -184, 18, 46)

## Floor Divide (Actually Round Divide)

In [17]:
print(h1)
print(h4)
print(h1 // h4)

(46 + -92i + 9j + 23k)
(-92 + 33i + 25j + -17k)
(-1 + 1i + 0j + 0k)


## Modified DivMod

In [18]:
help(Hi.modified_divmod)

Help on function modified_divmod in module __main__:

modified_divmod(a, b)
    Returns q & r, such that a = b * q + r, where
    r.norm < b.norm / 2



In [19]:
a = h1
b = h4
print(a)
print(b)
quot, rem = Hi.modified_divmod(a, b)
print(quot)
print(rem)

(46 + -92i + 9j + 23k)
(-92 + 33i + 25j + -17k)
(-1 + 1i + 0j + 0k)
(-13 + 33i + 51j + 31k)


In [20]:
print(f"{h4 * quot + rem}\n = {h4}\n   * {quot}\n   + {rem}")

(46 + -92i + 9j + 23k)
 = (-92 + 33i + 25j + -17k)
   * (-1 + 1i + 0j + 0k)
   + (-13 + 33i + 51j + 31k)


In [21]:
print(rem.norm)
print(b.norm)
print(rem.norm < b.norm / 2)

4820
10467
True
